# This Notebook provides the walk through each step for single sleep-EEG (".edf") file to present some details about possible the package in details - Part2.

The whole notebooks parts should be run together inorder to explain the first each note-books are provided seperately.
This will help the user to identify the any-issues while first time running the package.

However these parts include the first part and wanted functions in the first block.
So the user can run the first block as whole to finish the previous parts and check the next stages.

If the user want to run the whole script together pleace check the last-note-book.

## Upto part-1 and needed modules

Since you are using the script to run the file, please load the latest .whl file 
located in the "EEG_PSG_loose_lead_test/dist/sleep_EEG_loose_lead_detect-0.0-py3-none-any.whl"

Once the package is dowloaded, then change the directory via the command in cmd-prompt

cd .../EEG_PSG_loose_lead_test/dist/

Then install the packge via the pip command via the command in cmd-prompt,

pip install sleep_EEG_loose_lead_detect-0.0-py3-none-any.whl

In [ ]:
import os
import sys
import logging

import numpy as np
import pandas as pd
import pickle


from copy import deepcopy
from collections import Counter
# from multiprocessing import Pool#, TimeoutError

import sleep_EEG_loose_lead_detect as sleep_EEG_loose_lead_detect
# --------------------------------------------------------------------------
# load package functions
# assign the working directory of the package
# --------------------------------------------------------------------------
from sleep_EEG_loose_lead_detect.directory_utils import EEG_sleep_dir

from sleep_EEG_loose_lead_detect.optional_parameters import parameter_assignment

#checking only one get_root_channels function whether the loading suceeds
# # here non-normlaised EEG is obtained in the preproceeing step in time domain
from sleep_EEG_loose_lead_detect.preprocessing.load_EDF_dataset_events import load_root_dataset

from sleep_EEG_loose_lead_detect.preprocessing.find_bad_epochs import markBadEpochs
from sleep_EEG_loose_lead_detect.preprocessing.segment_filter_EEG import segment_EEG

from sleep_EEG_loose_lead_detect.loose_lead_events.events_to_txt import preprocess_events_to_txt

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger("sleep_loose_lead")
while logger.handlers:
      logger.handlers.pop()
c_handler = logging.StreamHandler()
# link handler to logger
logger.addHandler(c_handler)
logger.setLevel(logging.INFO)
logger.propagate = False



'''
direct_run makes skip the user inputs
'''
loading_dir_pre =EEG_sleep_dir(splidle_inc=False)
# --------------------------------------------------------------------------
#  first assign the i/p and o/p directories
# --------------------------------------------------------------------------
loading_dir_pre.in_loc = '/Users/anandanadarajn/Documents/.../results_pickles/check_loose_lead/'
loading_dir_pre.out_loc ='/Users/anandanadarajn/Documents/.../results_pickles/check_loose_lead/'

loading_dir_pre.keep_signature_dic ={}
loading_dir_pre.keep_signature_dic['dic']=True
loading_dir_pre.keep_signature_dic['evtxt']=True
loading_dir_pre.keep_signature_dic['bad_epochs']=True
loading_dir_pre.keep_signature_dic['out_loc_outlier']=True
loading_dir_pre.keep_signature_dic['sleep_anot']=True
loading_dir_pre.keep_signature_dic['MT_spec']=True
loading_dir_pre.keep_signature_dic['annota_NREM_REM']=True
loading_dir_pre.keep_signature_dic['splidle_inc']=True
loading_dir_pre.keep_signature_dic['tex_files']=True

loading_dir_pre.save_spindle_loc = True

# --------------------------------------------------------------------------
# re assign the loading_dir_pre object
# --------------------------------------------------------------------------
loading_dir_pre.assign_directories()
f='edf_name'

'''
getting user input and call loose-lead detection
'''
opt_paramters = parameter_assignment()
opt_paramters.loading_dir_pre = loading_dir_pre

opt_paramters.sep = False

opt_paramters.tag='_def'

Then load the intial parameters and assign the edf name

In [ ]:
pred_slow_waves = opt_paramters.pred_slow_waves
pred_spindles = opt_paramters.pred_spindles 

T = opt_paramters.T
amplitude_high_same_all_age = opt_paramters.amplitude_high_same_all_age
avoid_spindle_loc = opt_paramters.avoid_spindle_loc
verbose  = opt_paramters.verbose


# --------------------------------------------------------------------------
# assigning the directories
# --------------------------------------------------------------------------
in_loc = loading_dir_pre.in_loc
# save_converted_index_loc = loading_dir_pre.save_converted_index_loc
# out_loc_outlier = loading_dir_pre.out_loc_outlier
in_bad_events = loading_dir_pre.bad_events_file_path
save_spindle_loc = loading_dir_pre.save_spindle_loc


# --------------------------------------------------------------------------
#  if the file not ends with edf add edf extention
# --------------------------------------------------------------------------

if in_name_temp.split('.')[-1]!='edf':
    in_edf = in_loc + in_name_temp + ".edf"
else:
    in_edf = in_loc + in_name_temp 
    in_name_temp = '.'.join(in_name_temp.split('.')[0:-1])
logger.warning('*** filename: %s', in_edf)

if len(opt_paramters.tag)>0:
    in_name_temp= in_name_temp+opt_paramters.tag

Get the parameters for the loading the raw-EEG data.

amplitude_thres = 2000 is default 
500 is checked in the batch_run_server_example_2.py


In [ ]:
epoch_length, line_freq, bandpass_freq, normal_only, notch_freq_essential_checker, amplitude_thres = opt_paramters.preprocess_par()

# --------------------------------------------------------------------------
# developments optional to user
# --------------------------------------------------------------------------
save_events_origin = opt_paramters.save_events_origin
sleep_stage_preprocess_origin = opt_paramters.sleep_stage_preprocess_origin

## Load the EEG raw data and events

Extract the epoch annotations (events) come along with the bigger edf. And only select the wanted channels' signal from the ``.edf''.

In [ ]:
os.chdir('/')#to make sure appending paths
EEG_root, sleep_stages_or, EEG_channels, Fs, start_time_idx,  sel_index_sl_st, whole_annotations, ev_or = load_root_dataset(in_edf, epoch_length)

# --------------------------------------------------------------------------
# save the events for checking purpose
# --------------------------------------------------------------------------
if save_events_origin:
    np.save(loading_dir_pre.out_loc_NREM_REM+ in_name_temp + "_sel_index_sl_st",sel_index_sl_st)
    np.save(loading_dir_pre.out_loc_NREM_REM+ in_name_temp + "_ev_or",ev_or)


## Mark the bad epoches (extract the annotated events)
Some times edfs have rich information other than the sleep-stages
like patients out for bathroom break, restroom, etc. The epoches fell into the bathroom break is just noise
so these kind of noise present data supposed to be removed from the sleep-stages annotation
    
However each technician use their own terminolgy. 
we haven't remove all the bad-events here, just some known events that is surely known as bad-events
we have provided the bad events in the /docs/bad_events_bathroom.txt files that can be updatable
    
some potential events are purposefully leaved like movenments, arousal etc. Since these information can be later 
used to find the subjects disease based studies etc.

In [ ]:

bad_epochs = markBadEpochs(in_edf, in_bad_events, epoch_sec=epoch_length)
if loading_dir_pre.keep_signature_dic['bad_epochs']:
    np.save(loading_dir_pre.bad_epochs_folder+ in_name_temp + "_bad_epochs",bad_epochs)

## Segment the stages into epoches
This is mainly focused on preprocess the EEG signal in the time domain and detect the anomalies (overly high amplitude signals) presented in the time domain.
In this part return the splited epochs signals. 
Filtered signals (the filterize remove powerline noise, etc.)

In [ ]:
#  make deep copy 
EEG_root_copy=deepcopy(EEG_root)
if avoid_spindle_loc:
    sleep_stages_origin =  deepcopy(sleep_stages_or)


sleep_stages=  deepcopy(sleep_stages_or)
# --------------------------------------------------------------------------
# Segment EEG into 30sec epochs, apply notch & band filters, mark bad epochs and normalization
# this is preprocessing the EEG signal in time domain and return the preprocessed signal in time domain
# --------------------------------------------------------------------------

# --------------------------------------------------------------------------
# this will force the events with the channels information
#   channel_specific_preprocess=True 
# if not channel_specific_preprocess just return the epoch status without channel specific information
# like nan value, high/lower amplitude etc.
# --------------------------------------------------------------------------
channel_specific_preprocess = opt_paramters.channel_specific_preprocess
# --------------------------------------------------------------------------
# EEG_channels extracted from the load_root_dataset
# such that filaly the default channels will be endup in
# ch_names = ['F3', 'F4', 'C3', 'C4', 'O1', 'O2']
# --------------------------------------------------------------------------
ch_names = EEG_channels    

# --------------------------------------------------------------------------
# in the fuiltertaion process the 
# EEG_root_copy have higher chance of filetisation
#  so keep them in a seperate deep copy is prefered
# 
#  to feed the filterd EEG through the MT-make sure assign this as True
# basically the epoches points to the EEG_root_copies location
# --------------------------------------------------------------------------
filtered_EEG_MT=True
epochs, EEG_root_copy, sleep_stages, epoch_start_idx_o, epoch_status, q1,q2,q3, notch_filter_skipped = segment_EEG(EEG_root_copy, 
                                                                                sleep_stages, 
                                                                                epoch_length, 
                                                                                Fs, start_time_idx, 
                                                                                notch_freq=line_freq, 
                                                                                bandpass_freq=bandpass_freq, 
                                                                                amplitude_thres=amplitude_thres, 
                                                                                channel_specific_preprocess=channel_specific_preprocess,
                                                                                ch_names=ch_names,
                                                                                bad_epochs=bad_epochs,notch_freq_essential_checker=notch_freq_essential_checker,
                                                                                return_filtered_EEG=True,
                                                                                verbose=verbose)

## Save the preprocess outcomes of epochs

Inorder to save the preprocess outcomes.

In [ ]:
preprocess_events_to_txt(in_name_temp+'_pre_process.txt',loading_dir_pre.out_loc_txt, epoch_status,  preprocess_txt_only_detected_bad=True)

## temp 

The temp dictionary is an option to save the intermediate extracted information, this will avoid re-computation.
When we need the eeg's function retrieval outcomes and information such as sampling frequency (Fs),  quantiles (Q1,Q2, and Q3), etc.

In [ ]:
temp= {}
temp['Fs']=Fs

temp['q1']=np.array(q1)
temp['q2']=np.array(q2)
temp['q3']=np.array(q3)

temp['notch_filter_skipped']=notch_filter_skipped

## Extracted epochs length check

This portion check the length of the extractec epochs, and check the number of useful (normal/ good_ids) time domain EEG signal for further processing. Here if the length of the good-ids are less than 300 epochs also that also saved in the ``temp'' dictionary.

In [ ]:
temp['empty_EEG_segs']=False
if epochs.shape[0] <= 0:
    temp['empty_EEG_segs']=True
    raise ValueError('Empty EEG segments')

if normal_only:
    good_ids = np.where(epoch_status=='normal')[0]
    sel_id_name='good_ids'
    if len(good_ids)<=300:
        temp['less_300']=True
    else:
        temp['less_300']=False
else:
    raise Exception("need to modify the epoch status to selecting status, sin cethe outlier detection is build based only on normal epoch in mind")